In [1]:
import os
import csv

import pandas as pd
import numpy as np

#from lib.stats import *
from stats import * 


import matplotlib.pyplot as plt
import seaborn as sns
from mrcnn import visualize

from sklearn.metrics import r2_score
from scipy.stats import pearsonr

from library_dicom.dicom_processor.tools.folders import * 

In [2]:
csv_pred_path = '/media/oncopole/DD 2To/SEGMENTATION/inference/prediction_dataset.csv'
df = pd.read_csv(csv_pred_path)

In [3]:
df.head()

,patient_id,study_id,study,ct_img,pet_img,mask_img,pred_img
0,53011101841003,1.2.250.1.199.10.90001.93080.20120515093605,ahl,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DD 2To/SEGMENTATION/inference/...
1,53011101881011,1.2.250.1.199.10.90001.97031.20130222102728,ahl,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DD 2To/SEGMENTATION/inference/...
2,53011101841011,1.2.250.1.199.10.90001.97274.20140120115149,ahl,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DDArchive/AHL_NIFTI/5301110184...,/media/oncopole/DD 2To/SEGMENTATION/inference/...
3,53011101881007,1.2.250.1.199.10.90001.98250.20120824131623,ahl,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DDArchive/AHL_NIFTI/5301110188...,/media/oncopole/DD 2To/SEGMENTATION/inference/...
4,53011101871009,1.2.250.1.38.2.1.102.130708113824.102392945,ahl,/media/oncopole/DDArchive/AHL_NIFTI/5301110187...,/media/oncopole/DDArchive/AHL_NIFTI/5301110187...,/media/oncopole/DDArchive/AHL_NIFTI/5301110187...,/media/oncopole/DD 2To/SEGMENTATION/inference/...


In [9]:
study = []
number = []
for idx,name in enumerate(df['study'].value_counts().index.tolist()):
    #print('Study:', name)
    study.append(name)
    #print('Number :', df['study'].value_counts()[idx])
    number.append(df['study'].value_counts()[idx])
#print(study)
#print(number)

percent = []
for s, n in zip(study, number):
    print(s)
    print("{} % ".format(round(n/np.sum(number)*100,2)))
    print('')
    percent.append(round(n/np.sum(number)*100,2))

ahl
29.54 % 

gained
24.12 % 

pvab
20.05 % 

relevance
11.92 % 

remarc
7.59 % 

flip
4.88 % 

lnh073b
1.9 % 



In [3]:
dataset = df.values.tolist()
dataset[0]

['53011101841003',
 '1.2.250.1.199.10.90001.93080.20120515093605',
 'ahl',
 '/media/oncopole/DDArchive/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_CT.nii',
 '/media/oncopole/DDArchive/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_PT.nii',
 '/media/oncopole/DDArchive/AHL_NIFTI/53011101841003/pet0/1.2.250.1.199.10.90001.93080.20120515093605_nifti_mask_3d.nii',
 '/media/oncopole/DD 2To/SEGMENTATION/inference/nifti/1.2.250.1.199.10.90001.93080.20120515093605_inference_pred.nii']

In [10]:
dataset[125]

['2017064',
 '2.16.840.1.113669.632.20.310000.20002776668',
 'flip',
 '/media/oncopole/DDArchive/FLIP_NIFTI_COMPLET/CR,064/pet0/2.16.840.1.113669.632.20.310000.20002776668_nifti_CT.nii',
 '/media/oncopole/DDArchive/FLIP_NIFTI_COMPLET/CR,064/pet0/2.16.840.1.113669.632.20.310000.20002776668_nifti_PT.nii',
 '/media/oncopole/DDArchive/FLIP_NIFTI_COMPLET/CR,064/pet0/2.16.840.1.113669.632.20.310000.20002776668_nifti_mask_3d.nii',
 '/media/oncopole/DD 2To/SEGMENTATION/inference/nifti/2.16.840.1.113669.632.20.310000.20002776668_inference_pred.nii']

In [4]:
len(dataset)

369

In [4]:
#DICE 
seuil = [0.41, 2.5, 4.0]
for inference in dataset[361:] :
    print(dataset.index(inference)) 
    pet_array, spacing = get_array_from_nifti(inference[4])
    for s in seuil : 
        if s == 0.41 : 
            #Watershed
            ws_array = applied_watershed_on_inference(inference[6], inference[4])
            pred_array = np.transpose(ws_array, (2,1,0))
            true_array, _ = get_array_from_nifti(inference[5]) 
            dice = calcul_dice(pred_array, true_array, pet_array, thresh = s)
            inference.append(dice[0])
            filename = inference[1]+'_'+str(s)
            write_json_file('/media/oncopole/DD 2To/SEGMENTATION/inference/stats/dice', filename, inference)
            print(dice)
            
        else : 
            
            pred_array, _ = get_array_from_nifti(inference[6])
            true_array, _ = get_array_from_nifti(inference[5])
            dice = calcul_dice(pred_array, true_array, pet_array, thresh = s)
            inference.append(dice[0])
            filename = inference[1]+'_'+str(s)
            write_json_file('/media/oncopole/DD 2To/SEGMENTATION/inference/stats/dice', filename, inference)
            print(dice)
           

361
[0.75858903]
[0.73418282]
[0.95843592]
362
[0.88229273]
[0.79605372]
[0.89711431]
363
[0.75215577]
[0.70013245]
[0.76092416]
364
[0.7582078]
[0.64849414]
[0.8305619]
365
[0.50539221]
[0.57132648]
[0.89666137]
366
[0.05695612]
[0.74048443]
[0.]
367
[0.60384741]
[0.6145653]
[0.87496732]
368
[0.84747454]
[0.78437293]
[0.92022923]


In [4]:
#TMTV
seuil = [0.41, 2.5, 4.0]
for inference in dataset[361:] : 
    print(dataset.index(inference))
    pet_array, spacing = get_array_from_nifti(inference[4])
    for s in seuil : 
        if s == 0.41 : 
            #Watershed
            ws_array = applied_watershed_on_inference(inference[6], inference[4])
            pred_array = np.transpose(ws_array, (2,1,0))
            true_array, _ = get_array_from_nifti(inference[5]) 
            tmtv_pred, tmtv_true = calcul_tmtv(pred_array, true_array, pet_array, spacing, thresh = s)
            inference.append(tmtv_pred)
            inference.append(tmtv_true)
            filename = inference[1]+'_'+str(s)
            write_json_file('/media/oncopole/DD 2To/SEGMENTATION/inference/stats/tmtv', filename, inference)
            #print("tmtv_pred :", tmtv_pred)
            #print("tmtv_true :", tmtv_true)
            
        else : 
            
            pred_array, _ = get_array_from_nifti(inference[6])
            true_array, _ = get_array_from_nifti(inference[5])
            tmtv_pred, tmtv_true = calcul_tmtv(pred_array, true_array, pet_array, spacing, thresh = s)
            inference.append(tmtv_pred)
            inference.append(tmtv_true)
            filename = inference[1]+'_'+str(s)
            write_json_file('/media/oncopole/DD 2To/SEGMENTATION/inference/stats/tmtv', filename, inference)
            #print("tmtv_pred :", tmtv_pred)
            #print("tmtv_true :", tmtv_true)
           


361
362
363
364
365
366
367
368


In [8]:
#csv_dice 

directory = '/media/oncopole/DD 2To/SEGMENTATION/inference/stats/dice'
liste_json = os.listdir(directory)
full_liste = []
for json_file in liste_json : 
    if '_4.0' in json_file : 
        full_liste.append(os.path.join(directory, json_file))

print(len(full_liste)) #must be 369
dataset = []
for json_file in full_liste : 
    with open(json_file) as json_file : 
        reader = json.load(json_file)
        dataset.append(reader)
print(dataset[0])
filename = 'inference_dice_dataset.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["patient_id", "study_uid", "study", "ct_img", "pet_img", "mask_img", "pred_img", "dice_0.41", "dice_2.5", "dice_4.0"])
    for serie in dataset: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6], serie[7], serie[8], serie[9]])

369
['13014101433004', '1.2.276.0.7230010.3.1.2.413926400.1.1607332253.423543', 'pvab', '/media/oncopole/DDArchive/PVAB_NIFTI/13014101433004/PET0/1.2.276.0.7230010.3.1.2.413926400.1.1607332253.423543_nifti_CT.nii', '/media/oncopole/DDArchive/PVAB_NIFTI/13014101433004/PET0/1.2.276.0.7230010.3.1.2.413926400.1.1607332253.423543_nifti_PT.nii', '/media/oncopole/DDArchive/PVAB_NIFTI/13014101433004/PET0/1.2.276.0.7230010.3.1.2.413926400.1.1607332253.423543_nifti_mask_3d.nii', '/media/oncopole/DD 2To/SEGMENTATION/inference/nifti/1.2.276.0.7230010.3.1.2.413926400.1.1607332253.423543_inference_pred.nii', 0.22711320565526924, 0.7003100241129866, 0.9130434782608695]


In [9]:
#csv_tmtv

directory = '/media/oncopole/DD 2To/SEGMENTATION/inference/stats/tmtv'
liste_json = os.listdir(directory)
full_liste = []
for json_file in liste_json : 
    if '_4.0' in json_file : 
        full_liste.append(os.path.join(directory, json_file))

print(len(full_liste)) #must be 369
dataset = []
for json_file in full_liste : 
    with open(json_file) as json_file : 
        reader = json.load(json_file)
        dataset.append(reader)
filename = 'inference_tmtv_dataset.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["patient_id", "study_uid", "study", "ct_img", "pet_img", "mask_img", "pred_img", "pred_0.41", "true_0.41", "pred_2.5", "true_2.5", "pred_4.0", "true_4.0"])
    for serie in dataset: 
        csv_writer.writerow([serie[0], serie[1], serie[2], serie[3], serie[4], serie[5], serie[6], serie[7], serie[8], serie[9], serie[10], serie[11], serie[12]])

369


In [ ]:
result_csv_path = '/home/oncopole/Documents/Rudy/DeepOncopole/data/vnet/dice3d_results.csv'

In [ ]:
df = pd.read_csv(result_csv_path,
                sep="\t|,",)

In [ ]:
df.info()

In [ ]:
df_train = df[df['subset']=='train'].copy()
df_val = df[df['subset']=='val'].copy()
df_test = df[df['subset']=='test'].copy()

In [ ]:
df_train.describe()

In [ ]:
df_train.median()

In [ ]:
df_val.describe()

In [ ]:
df_val.median()

In [ ]:
ax = sns.boxplot(x="subset", y="dice_cnn", data=df)
ax = sns.swarmplot(x="subset", y="dice_cnn", data=df, color=".25")

# ax.set_ylim(0.0, 1.0)
plt.show()

In [ ]:
plot_result(df_train)

In [ ]:
plot_result(df_val)

In [ ]:
plot_result(df_test)